In [1]:
!pip install finance-datareader

In [2]:
import FinanceDataReader as fdr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from datetime import datetime, timedelta
import random

num_images = 5000
day_num = 32

In [3]:
print(len(fdr.DataReader("005930", '2022-12-31')))

152


In [4]:
def convert_image(data, num_images, day_num, change_std, save = False, test = False):
    if test:
        change_std = 0

    def alpha_blend(color1, color2, alpha):
        return [
            int((color1[0] * alpha + color2[0] * (1 - alpha))),
            int((color1[1] * alpha + color2[1] * (1 - alpha))),
            int((color1[2] * alpha + color2[2] * (1 - alpha))),
            255,
        ]

    # 데이터수가 너무 적으면 패스
    if len(data) <= 100:
        return []


    # 해당 종목의 주가 데이터 - 100이 요구되는 이미지 수보다 적으면(상장직후는 변동이 심할 수있으니 제외)
    if num_images - 100 > len(data):
        num_images = len(data) - 100

    #return_datas는 이미지, 라벨링(상승이면1 하락이면0), 변동률, 날짜를 담고있음
    return_datas = []
    prediction_day_num = 5


    image_idx = 1
    while(image_idx <= num_images):
        prediction_day = data[-image_idx - prediction_day_num : -image_idx]
        predict_mean = prediction_day['Close'].mean()
        now = data.iloc[-image_idx - prediction_day_num - 1]
        change = (predict_mean - now['Close']) / now['Close']


        # change가 너무 높으면 이상치라고 판단하여 패스
        if not test and (abs(change) < change_std or change_std * 2 < abs(change)): # 향후 prediction_day_num일의 변동률이 기준 이하면
            image_idx += 1
            continue

        stock_data = data[-image_idx - day_num - prediction_day_num : -image_idx - prediction_day_num]

        # 이미지 크기 설정
        width, height = 96, 96

        # High, Low, Close, Volume 데이터 정규화
        high_prices = stock_data['High'].values
        low_prices = stock_data['Low'].values
        close_prices = stock_data['Close'].values
        volume_data = stock_data['Volume'].values

        high_prices_norm = (high_prices - np.min(low_prices)) / (np.max(high_prices) - np.min(low_prices))
        low_prices_norm = (low_prices - np.min(low_prices)) / (np.max(high_prices) - np.min(low_prices))
        close_prices_norm = (close_prices - np.min(low_prices)) / (np.max(high_prices) - np.min(low_prices))
        volume_data_norm = height * (volume_data) / (2 * (np.max(volume_data)))

        # 이동평균선 추가를 위한 stock_data 생성
        stock_data_ma20 = data[-image_idx - day_num - 20 - prediction_day_num: -image_idx - prediction_day_num]
        stock_data_ma20 = stock_data_ma20.copy()
        stock_data_ma20['MA20'] = stock_data_ma20['Close'].rolling(window=20).mean()
        stock_data_ma20.dropna(inplace=True)

        # 이동평균선 값을 정규화
        stock_data_ma20['MA20_norm'] = (
                (stock_data_ma20['MA20'] - np.min(low_prices)) / (np.max(high_prices) - np.min(low_prices))
            )

        # 96x96 이미지 생성
        img = np.zeros((height, width, 4), dtype=np.uint8)  # 4 channels for RGBA

        fail_generate = False
        for day in range(32):
            high_price = int(high_prices_norm[day] * (height - 1))
            low_price = int(low_prices_norm[day] * (height - 1))
            close_price = int(close_prices_norm[day] * (height - 1))
            open_price = int(
                (
                    (stock_data['Open'].values[day] - np.min(low_prices))
                    / (np.max(high_prices) - np.min(low_prices))
                )
                * (height - 1)
            )
            volume = int(volume_data_norm[day])
            if(volume == 0):
                fail_generate = True
                break

            x_start = day * 3

            # 시가, 종가, 고가, 저가 막대그래프
            # 막대 왼쪽, 오른쪽 그리기 (시가, 종가 범위)
            for x in range(x_start, x_start + 3):
                if close_prices[day] > stock_data['Open'].values[day]:
                    color = [255, 0, 0, 255]  # 상승: 빨강
                    tail_color = [200, 0, 0, 255]  # 상승 시 꼬리: 어두운 빨강
                elif close_prices[day] < stock_data['Open'].values[day]:
                    color = [0, 0, 255, 255]  # 하락: 파랑
                    tail_color = [0, 0, 200, 255]  # 하락 시 꼬리: 어두운 파랑
                else: # 시가와 종가가 같을경우 어제보다 같거나 올랐으면 빨강, 아니면 파랑
                    if 0 <= stock_data['Change'].values[day]:
                        color = [255, 0, 0, 255]  # 상승: 빨강
                        tail_color = [200, 0, 0, 255]  # 상승 시 꼬리: 어두운 빨강
                    else:
                        color = [0, 0, 255, 255]  # 하락: 파랑
                        tail_color = [0, 0, 200, 255]  # 하락 시 꼬리: 어두운 파랑

                for y in range(min(open_price, close_price), max(open_price, close_price) + 1):
                    img[height - 1 - y, x] = color

            # 꼬리 그리기 (High, Low 범위)
            for x in range(x_start + 1, x_start + 2):
                for y in range(low_price, high_price + 1):
                    if img[height - 1 - y, x][0] != 255 and img[height - 1 - y, x][2] != 255:
                        img[height - 1 - y, x] = tail_color

            # 거래량 그리기
            for x in range(x_start, x_start + 3):
                overlapped = height - volume
                if overlapped >= 0 and overlapped < height:
                    img[overlapped:, x] = [
                        alpha_blend(img[y, x], [128, 128, 128, 255], 0.45)
                        for y in range(overlapped, height)
                    ]

            # 20일 이동평균선 그리기
            ma20_norm = stock_data_ma20.iloc[day]['MA20_norm']

            for x in range(x_start, x_start + 3):
                if not np.isnan(ma20_norm):
                    ma20_y = int(ma20_norm * (height - 1))
                    if 0 <= ma20_y < height:
                        for y in range(ma20_y - 1, ma20_y + 2):  # 이동평균선 주변에 블렌딩 처리
                            if 0 <= y < height:
                                new_color = alpha_blend([255, 255, 0, 255], img[height - 1 - y, x], 0.6)  # 노란색으로 변경
                                img[height - 1 - y, x] = new_color

        #이미지가 day_num(그려지기 원하는 거래일 수)미만으로 그려졌으면
        if fail_generate:
            image_idx += prediction_day_num
            continue

        if save :
            # 이미지 저장
            img = Image.fromarray(img, 'RGBA')
            img.save(f'samsung_stock_data_image_{image_idx}.png')

        if change_std <= change:
            return_datas.append((img, 1))
        elif change <= -change_std :
            return_datas.append((img, 0))

            # 테스트는 촘촘하게 모든 데이터로
        if test:
            image_idx += random.choice([1, 2, 3])
        else:
            # 비슷한 이미지를 피하기 위해 기준 예측일만큼 건너뜀
            image_idx += prediction_day_num


    return return_datas

In [5]:
def preprocess_data(data):
    # 0이나 NaN을 포함하는 행 제거
    data = data[data['Volume'] != 0]
    data = data.dropna(how='any')

    return data

In [6]:
stock_codes = [
    ["005930", "삼성전자"],
    ["051910", "LG화학"],
    ["000660", "SK하이닉스"],
    ["207940", "삼성바이오로직스"],
    ["005935", "삼성전자우"],
    ["051915", "LG화학우"],
    ["006400", "삼성SDI"],
    ["005380", "현대차"],
    # ["035420", "NAVER"],
    ["363280", "효성화학"],
    ["247540", "에코프로비엠"],
    ["000270", "기아"],
    ["086520", "에코프로"],
    ["035720", "카카오"],
    ["068270", "셀트리온"],
    ["012330", "현대모비스"],
    ["105560", "KB금융"],
    ["028260", "삼성물산"],
    ["055550", "신한지주"],
    ["096770", "SK이노베이션"],
    ["066570", "LG전자"],
    ["047050", "대우조선해양"],
    ["032830", "삼성생명"],
    ["323410", "카카오뱅크"],
    ["003550", "LG"],
    ["015760", "한국전력"],
    ["000810", "삼성화재"],
    ["033780", "KT&G"],
    ["091990", "셀트리온헬스케어"],
    ["009150", "삼성전기"],
    ["086790", "하나금융지주"],
    ["034730", "SK"],
    ["336370", "솔루스첨단소재"],
    ["352820", "하이브"],
    ["018260", "삼성에스디에스"],
    ["122630", "KODEX 레버리지"],
    ["017670", "SK텔레콤"],
    ["010130", "고려아연"],
    ["024880", "케이씨케미칼"],
    ["003490", "대한항공"],
    ["010950", "S-Oil"],
    ["086280", "현대글로비스"],
    ["259960", "크래프톤"],
    ["024110", "기업은행"],
    ["030200", "케이티"],
    ["316140", "우리금융지주"],
    ["010140", "삼성중공업"],
    ["090430", "아모레퍼시픽"],
    ["051900", "LG생활건강"],
    ["028050", "삼성엔지니어링"],
    ["233740", "KODEX 코스닥 150 레버리지"],
    ["011170", "롯데케미칼"],
    ["000100", "유한양행"],
    ["012450", "한화에어로스페이스"],
    ["036570", "엔씨소프트"],
    ["241560", "두산밥캣"],
    ["005830", "DB손해보험"],
    ["018880", "한온시스템"],
    ["034220", "LG디스플레이"],
    ["170030", "HD현대"],
    # ["161390", "한국타이어앤테크놀로지"],
    ["088980", "맥쿼리인프라"],
    ["005070", "코스모신소재"],
    ["271560", "오리온"],
    ["004020", "현대제철"],
    ["097950", "CJ제일제당"],
    ["047810", "한국항공우주"],
    ["042700", "한미반도체"],
    ["032640", "LG유플러스"],
    ["307950", "현대오토에버"],
    ["006800", "미래에셋증권"],
    ["007700", "F&F"],
    ["251270", "넷마블"],
    ["128940", "한미약품"],
    ["000720", "현대건설"],
    ["010620", "현대미포조선"],
    ["006260", "LS"],
    ["078930", "GS"],
    ["011780", "금호석유"],
    ["035250", "강원랜드"],
    ["008770", "호텔신라"],
    ["005940", "NH투자증권"],
    ["180640", "한진칼"],
    ["016360", "삼성증권"],
    ["064350", "현대로템"],
    ["021240", "코웨이"],
    ["282330", "BGF리테일"],
    ["112610", "씨에스윈드"],
    ["071050", "한국금융지주"],
    ["002790", "아모레G"],
    ["008930", "한미사이언스"],
    ["004990", "롯데지주"],
    ["010780", "쌍용C&E"],
    ["001450",  "현대해상"],
    ["111770",  "영원무역"],
    ["272210",  "한화시스템"],
    ["052690",  "한전기술"],
    ["267260",  "HD현대일렉트릭"],
    ["007070",  "GS리테일"],
    ["028670",  "팬오션"],
    ["139480",  "이마트"]
]
print(len(stock_codes))

99


In [7]:
train_result = []
for i, ticker in enumerate(stock_codes):
    code, name = ticker[0], ticker[1]
    print(f"{i+1}번째 종목 {name} 추출중...   ")

    data = preprocess_data(fdr.DataReader(code, '1999-01-01', '2022-12-31'))  # 각 종목의 데이터 가져오기
    images = convert_image(data, 5000, day_num, 0.05)
    train_result += images
    print(f"{len(images)}개 추출완료")

train_images, train_labels = zip(*train_result)
np.save('train_images.npy', train_images)
np.save('train_labels.npy', train_labels)

1번째 종목 삼성전자 추출중...   
184개 추출완료
2번째 종목 LG화학 추출중...   
317개 추출완료
3번째 종목 SK하이닉스 추출중...   
381개 추출완료
4번째 종목 삼성바이오로직스 추출중...   
85개 추출완료
5번째 종목 삼성전자우 추출중...   
205개 추출완료
6번째 종목 LG화학우 추출중...   
314개 추출완료
7번째 종목 삼성SDI 추출중...   
321개 추출완료
8번째 종목 현대차 추출중...   


KeyboardInterrupt: ignored

In [ ]:
print(len(train_result))

In [ ]:
test_result = []
for i, ticker in enumerate(stock_codes):
    code, name = ticker[0], ticker[1]
    print(f"{i+1}번째 종목 {name} 추출중...   ")

    data = preprocess_data(fdr.DataReader(code, "2022-10-01"))  # 각 종목의 데이터 가져오기
    images = convert_image(data, 3000, day_num, 0.05, test=True)
    test_result += images
    print(f"{len(images)}개 추출완료")

test_images, test_labels = zip(*test_result)
np.save('test_images.npy', test_images)
np.save('test_labels.npy', test_labels)

In [ ]:
print(len(test_result))

In [ ]:
train_images = np.load('train_images.npy')
train_labels = np.load('train_labels.npy')
test_images = np.load('test_images.npy')
test_labels = np.load('test_labels.npy')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

images, labels = zip(*train_result)

# 이미지 개수와 그리드 크기 설정
num_images = len(images)
num_to_display = 100
grid_size = int(np.sqrt(num_to_display))

# 랜덤으로 이미지 인덱스 선택
random_indices = np.random.choice(num_images, num_to_display, replace=False)

fig, axes = plt.subplots(grid_size, grid_size, figsize=(16, 16))
for i in range(grid_size):
    for j in range(grid_size):
        img_idx = random_indices[i * grid_size + j]
        img = Image.fromarray(images[img_idx])
        axes[i, j].imshow(img)
        axes[i, j].set_xticks([])
        axes[i, j].set_yticks([])
        axes[i, j].set_xlabel(f'Img {img_idx + 1}', fontsize=9)

plt.subplots_adjust(wspace=0.1, hspace=0.1)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(10, 10, figsize=(16, 16))
for i in range(10):
    for j in range(10):
        img_idx = i * 10 + j
        img = Image.fromarray(test_images[img_idx])
        axes[i, j].imshow(img)
        axes[i, j].set_xticks([])
        axes[i, j].set_yticks([])
        axes[i, j].set_xlabel(f'Img {img_idx + 1}', fontsize=9)

plt.subplots_adjust(wspace=0.1, hspace=0.1)
plt.show()


In [ ]:
print(test_labels[:100])